In [1]:
!pip install groq python-dotenv numpy tqdm datasets

In [15]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [17]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [18]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm excited to help you with any math problems you may have. What's on your mind? Do you have a specific problem you're struggling with, or would you like me to suggest some examples to get us started?


#### GSM8K 데이터셋 확인해보기

In [19]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [20]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [21]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                predicted_answer = float(predicted_answer.replace(",", ""))
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [22]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [10]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [11]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:02<00:02,  2.43it/s]

Progress: [5/10]
Current Acc.: [20.00%]


100%|██████████| 10/10 [00:04<00:00,  2.12it/s]

Progress: [10/10]
Current Acc.: [30.00%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

In [15]:
# 0-shot
prompt_0 = construct_direct_prompt(0)
results_0, accuracy_0 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=prompt_0,
    num_samples=50
)
save_final_result(results_0, accuracy_0, "direct_prompting_0.txt")

# 3-shot
prompt_3 = construct_direct_prompt(3)
results_3, accuracy_3 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=prompt_3,
    num_samples=50
)
save_final_result(results_3, accuracy_3, "direct_prompting_3.txt")

# 5-shot
prompt_5 = construct_direct_prompt(5)
results_5, accuracy_5 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=prompt_5,
    num_samples=50
)
save_final_result(results_5, accuracy_5, "direct_prompting_5.txt")

 10%|█         | 5/50 [00:09<01:16,  1.71s/it]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:15<00:50,  1.26s/it]

Progress: [10/50]
Current Acc.: [40.00%]


 30%|███       | 15/50 [00:30<01:10,  2.00s/it]

...

Progress: [40/50]
Current Acc.: [17.50%]


 90%|█████████ | 45/50 [02:14<00:19,  3.97s/it]

Progress: [45/50]
Current Acc.: [15.56%]


100%|██████████| 50/50 [02:32<00:00,  3.05s/it]

Progress: [50/50]
Current Acc.: [16.00%]


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [23]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Solve each math problem step by step. Then state the final answer after 'Answer:'.\n" #TODO: 프롬프트를 작성해주세요!

    for i in range(num_examples):
        #TODO: CoT example을 만들어주세요!
        idx = sampled_indices[i]
        cur_question = train_dataset["question"][idx]
        full_answer = train_dataset["answer"][idx].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"{full_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

In [24]:
# 0-shot
cot_prompt_0 = construct_CoT_prompt(0)
cot_results_0, cot_accuracy_0 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=cot_prompt_0,
    num_samples=50
)
save_final_result(cot_results_0, cot_accuracy_0, "CoT_prompting_0.txt")

# 3-shot
cot_prompt_3 = construct_CoT_prompt(3)
cot_results_3, cot_accuracy_3 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=cot_prompt_3,
    num_samples=50
)
save_final_result(cot_results_3, cot_accuracy_3, "CoT_prompting_3.txt")

# 5-shot
cot_prompt_5 = construct_CoT_prompt(5)
cot_results_5, cot_accuracy_5 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=cot_prompt_5,
    num_samples=50
)
save_final_result(cot_results_5, cot_accuracy_5, "CoT_prompting_5.txt")

 10%|█         | 5/50 [00:02<00:21,  2.13it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:04<00:17,  2.35it/s]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [00:07<00:17,  1.97it/s]

...

Progress: [40/50]
Current Acc.: [65.00%]


 90%|█████████ | 45/50 [08:34<00:58, 11.67s/it]

Progress: [45/50]
Current Acc.: [66.67%]


100%|██████████| 50/50 [09:21<00:00, 11.23s/it]

Progress: [50/50]
Current Acc.: [66.00%]


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [27]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(example_list: List[str], num_examples: int = 3):
    # TODO: 구현해주세요!

    prompt = (
        "You are a brilliant math tutor helping students solve problems step-by-step.\n"
        "Fore each question:"
        "1. Think step by step using labeled steps (Step 1, Step 2, ...)."
        "2. Keep reasoning short and numerical when possible."
        "3. Always end with a final line in the format: 'Answer: ####'.\n"
        "Let's look at some examples:\n"
    )
    for example in example_list[:num_examples]:
        prompt += f"{example}\n\n"
    
    prompt += (
        "Now, solve the following question carefully.\n"
        "Question:\n{question}\n"
        "Let's think step by step:\n"
        "Answer:"
    )

    return prompt

example_list = [
    """Question:
    Tom buys 5 packs of pencils. Each pack has 6 pencils. How many pencils does he have in total?
    
    Step 1: Each pack = 6 pencils
    Step 2: 4 packs -> 5 x 6 = 30
    Answer: #### 30""",

    """Question:
    A small bag weighs 2 kg. A big bag weighs 3 kg more than the small bag. What is the weight of the big bag?
    
    Step 1: Small bag = 2 kg
    Step 2: Big bag = 2 + 3 = 5 kg
    Answer: #### 5""",

    """Question:
    A pencil costs $0.50. A notebook costs three times more than a pencil. How much do both items cost together?

    Step 1: A pencil costs $0.50.
    Step 2: A notebook costs 3 x $0.50 = $1.50.
    Step 3: Total = $0.50 + $1.50 = $2.00
    Answer: #### 2.0
    """,

    """Question:
    Tom reads 20 pages per day. After 5 days, how many pages has he read?
    
    Step 1: Tom reads 20 pages per day
    Step 2: After 5 days, he reads 20 x 5 = 100 pages
    Answer: #### 100""",

    """Question:
    A train travels 60 km per hour. How far does it go in 2.5 hours?
    
    Step 1: Speed = 60 km/hour
    Step 2: Time = 2.5 hours
    Step 3: Distance = 60 x 2.5 = 150 km
    Answer: #### 150""",

    """Question:
    A rectangle has a length of 10 cm and width of 4 cm. What is its area?
    
    Step 1: Area of a rectangle = length x width
    Step 2: 10 x 4 = 40
    Answer: #### 40"""
]

In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

In [29]:
# 0-shot
my_prompt_0 = construct_my_prompt(example_list, 0)
results_my_0, accuracy_my_0 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=my_prompt_0,
    num_samples=50
)
save_final_result(results_my_0, accuracy_my_0, "My_prompting_0.txt")

# 3-shot
my_prompt_3 = construct_my_prompt(example_list, 3)
results_my_3, accuracy_my_3 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=my_prompt_3,
    num_samples=50
)
save_final_result(results_my_3, accuracy_my_3, "My_prompting_3.txt")

# 5-shot
my_prompt_5 = construct_my_prompt(example_list, 5)
results_my_5, accuracy_my_5 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=my_prompt_5,
    num_samples=50
)
save_final_result(results_my_5, accuracy_my_5, "My_prompting_5.txt")

 10%|█         | 5/50 [00:04<00:37,  1.19it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:07<00:27,  1.44it/s]

Progress: [10/50]
Current Acc.: [80.00%]


 30%|███       | 15/50 [00:15<00:46,  1.32s/it]

...

Progress: [40/50]
Current Acc.: [72.50%]


 90%|█████████ | 45/50 [05:25<00:35,  7.13s/it]

Progress: [45/50]
Current Acc.: [75.56%]


100%|██████████| 50/50 [06:02<00:00,  7.25s/it]

Progress: [50/50]
Current Acc.: [76.00%]


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!